# Customizing Our Agent

In this exercise, you'll customize both your agent and its tool to improve **clarity**, **transparency**, and **structure**.

![Customizing your agent](images/agent_transparency.png)

### ❗️ Note: Run the **hidden cell** below to initialize the agent, before running the rest of the code. ❗️ 

In [3]:
!pip install -q haystack-ai

### 🛠️ Customizing the Tool

Override the tool's name and description by passing them directly to the `ComponentTool`.
Use `outputs_to_string` to format the list of documents before they're passed to the LLM, and `outputs_to_state` to store them in the agent’s internal state.

In [4]:
from haystack.tools import ComponentTool
from haystack.components.websearch import SerperDevWebSearch

def doc_to_string(documents) -> str:
    result_str = ""
    for document in documents:
        result_str += f"Content for {document.meta['link']}: {document.content}\n\n"
    return result_str

search_tool = ComponentTool(
    component=SerperDevWebSearch(top_k=5),
    name="web_search_tool",
    description="Search the web",
    ____={"source": "documents", "handler": doc_to_string}, 
    outputs_to_state={"documents": {"source": "documents"}}
)

### ⚙️ Configuring a Traceable Agent
Define `state_schema` to store the retrieved documents.
Use the built-in `print_streaming_chunk` callback to display tool calls and their results as the agent runs.

In [5]:
from haystack.components.generators.utils import print_streaming_chunk
from haystack.components.agents import Agent
from haystack.components.generators.chat import OpenAIChatGenerator
from haystack.dataclasses import ChatMessage, Document

agent = Agent(
    chat_generator=OpenAIChatGenerator(model="gpt-4o-mini"),
    system_prompt="""
    You are a helpful AI assistant that has access to internet.
    Keep your answer concise and use the tools that you're provided with to answer the user's questions.
    """,
    tools=[search_tool],
    ____={"documents":{"type":list[Document]}}, 
    streaming_callback=____ 
)

### ▶️ Running the Agent with Trace Logs
Run the agent with a query and watch the full execution trace:
tool selection, query parameters, document results, and the final LLM response.

In [6]:
agent_results = agent.run(messages=[ChatMessage.from_user("What are the common side effects of hyaluronic acid?")])

### 📄 Inspecting Saved Documents
The documents retrieved by the web tool were saved in the agent's internal state. Inspect them using the `documents` field of the output.

In [7]:
agent_results["documents"]